# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-Lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are only allowed to use **Python 3.6** for implementation.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **Lab2** via following link: https://kg.cse.unsw.edu.au/submit/

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Ed.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 18th March, 2021 (Sydney Time)**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (100 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

We have pre-defined the function `buc_rec_optimized` in the file `submission.py`, and its helper functions are defined in the file `helper.py`. 

**Note:** You should use the functions defined in the file `helper.py`, you are not allowed to change this file. We will provide this file in the test environment.

## Input and output

Both `input` and `output` are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [1]:
import pandas as pd
import numpy as np

In [2]:
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.

def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

In [3]:
def single_tuple_result_print_out(df):
    
    # if there is only one column left
    # which is the base case
    if df.shape[1] == 1:
        return df
    else:
        # the idea is to append dataframe
        # 1  2
        # 1  *
        # *  2
        # *  *
        first_dimension = df.columns[0]
        remain_df = df.drop(first_dimension, axis=1)
        inner_df = single_tuple_result_print_out(remain_df)
        inner_df_copy = inner_df.copy()
        # first df with column of value 1
        inner_df.insert(0, first_dimension, df.iloc[0][first_dimension])
        # second df with column of value *
        inner_df_copy.insert(0, first_dimension, df.iloc[1][first_dimension])

        result = pd.concat([inner_df, inner_df_copy], ignore_index=True)
    return result

def single_tuple_optimization(df):
    
# append a star row at the bottom 
# in order to do the simple_tuple_optimization
    num_of_stars = len(list(df))-1
    star_array = []
    for i in range (num_of_stars):
        star_array.append('*')
    # add a row of stars and M value at the last
    star_array.append(df.iloc[0][df.columns[-1]])
    df.loc['1'] = star_array

    # the dataframe without the 'M' column
    dimension_df = df.drop(df.columns[-1], axis=1)
    # get the result of single_tuple
    
    result = single_tuple_result_print_out(dimension_df)
    # Add a uniform M value to the last column
    result['M'] = df.iloc[0][df.columns[-1]]
    
    return result
#=================================================================================

def buc_rec_optimized(df):# do not change the heading of the function
    
    dimensionlist = list(df)[0:-1] # produce A, B
    
    result = pd.DataFrame(columns=list(df))
    
    for index, dimension in enumerate(dimensionlist):
        dimension_uniq_value_list = df[dimension].unique() # [1,2]
        for dim_value in dimension_uniq_value_list:
            dim_df = select_data(df, index, dim_value)
            print(dim_df)
            if dim_df.shape[0] == 1: # check whether it is a single tuple
                result = pd.concat([result, single_tuple_optimization(dim_df)], ignore_index=True)
            else:
                sliced_df = remove_first_dim(dim_df)
                sub_result = buc_rec_optimized(sliced_df)
                sub_result_copy = sub_result.copy()
                sub_result.insert(0, dimension, dim_value) # append column A
                
                sub_result_copy.insert(0, dimension, '*')
                
                partial_result = pd.concat([sub_result, sub_result_copy], ignore_index=True)
                
                result = pd.concat([result, partial_result], ignore_index=True)
        break

    aggregation_functions = {'M': 'sum'}
    df_new = result.groupby(dimensionlist,as_index=False).aggregate(aggregation_functions)
    
    return df_new

In [3]:
def single_tuple_optimization(df):

    M = df.columns[-1]
    df.loc['1'] = 'ALL' # add a row  with "ALL"
    df.iloc[1][M] = df.iloc[0][M]

    cross_product_dict = df.to_dict('list') #‘list’ : dict like {column -> [values]}
    cross_product_dict[M] = [df.iloc[0][M]] # M -> [one value (not two)]

    col_names = pd.MultiIndex.from_product(cross_product_dict.values(), names=cross_product_dict.keys())
    cross_product_res_df = pd.DataFrame(index=col_names).reset_index()

    return cross_product_res_df 

def buc_rec_optimized(df):

    df_num_rows = df.shape[0]
    df_num_cols = df.shape[1]

    if df_num_rows == 1: # single tuple optimization

        single_tuple_result = single_tuple_optimization(df)

        return single_tuple_result
    elif df_num_cols == 1: # Only M column -> aggregation

        agg_sum = sum(helper.project_data(df, 0))
        agg_df = pd.DataFrame([agg_sum], columns=[df.columns[0]])

        return agg_df
    else:

        dim_0_valset = set(helper.project_data(df, 0).values)
        dim_0_result_list = [] # store the childern dataframes for each val
        for val in dim_0_valset:
            df_sliced = helper.slice_data_dim0(df, val)
            sub_result = buc_rec_optimized(df_sliced)
            sub_result.insert(0, df.columns[0], val) # append A 1 to all childern 
            dim_0_result_list.append(sub_result)
            
        df_sliced = helper.remove_first_dim(df)
        sub_result = buc_rec_optimized(df_sliced)
        sub_result.insert(0, df.columns[0], 'ALL')
        dim_0_result_list.append(sub_result)
        
        result = pd.concat(dim_0_result_list, ignore_index=True)
        return result

In [8]:
## You can test your implementation using the following code...
import helper
import submission as submission
input_data = read_data('./asset/ass.txt')
output = submission.buc_rec_optimized(input_data)

/Applications/anaconda3/envs/COMP9318/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Applications/anaconda3/envs/COMP9318/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [9]:
output

,L,T,I,M
0,ALL,2005,ALL,3100
1,ALL,2005,ps2,1400
2,ALL,2005,xb,1700
3,ALL,2006,ALL,2000
4,ALL,2006,ps2,1500
5,ALL,2006,will,500
6,ALL,ALL,ALL,5100
7,ALL,ALL,ps2,2900
8,ALL,ALL,will,500
9,ALL,ALL,xb,1700
